In [1]:
"""Import libraries"""
import nltk
import re
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

nltk.download('stopwords')

/tmp/ipykernel_1326671/3176950661.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as  pd
/usr/lib/python3/dist-packages/paramiko/transport.py:237: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,
[nltk_data] Downloading package stopwords to /home/jcm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
"""Prepare stopwords"""

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [3]:
"""Load dataset"""
# LoadDataset
file_path = "./ExtractedArticles.csv"

df= pd.read_csv(file_path, index_col="Date", parse_dates=True)
df


,Title,Title_URL,Date.1,Text,Mental Health,suicide,opioids,overdose
Date,,,,,,,,
2023-12-20,"Pulse for the Week of December 25, 2023",https://www.enr.com/articles/57911-pulse-for-t...,20-Dec-23,Planning\n\nARIZONA\n\nExact Sciences is plann...,1,0,0,0
2023-11-09,Construction Nonfatal-Injury Illness Rate Dips...,https://www.enr.com/articles/57549-constructio...,9-Nov-23,The construction industry showed improvement l...,1,0,0,0
2023-10-12,NY Bridge Safety Summit Eyes Suicide Preventio...,https://www.enr.com/articles/57308-ny-bridge-s...,12-Oct-23,New York State Bridge Authority and Office of ...,1,9,0,0
2023-06-27,Hawaii Health Officials Sign Off on Military D...,https://www.enr.com/articles/56701-hawaii-heal...,27-Jun-23,A U.S. Dept. of Defense plan to begin the grav...,1,0,0,0
2023-06-15,"Pulse for the Week of June 19, 2023",https://www.enr.com/articles/56635-pulse-for-t...,15-Jun-23,Planning\n\nCONNECTICUT\n\nUnited Illuminating...,1,0,0,0
2023-05-03,"Mass., R.I. Stand Downs Raise Awareness On Men...",https://www.enr.com/articles/56377-mass-ri-sta...,3-May-23,After Shawn Nehiley injured his back on the jo...,1,2,2,2
2023-02-16,US Labor Secretary Marty Walsh Says He Will Le...,https://www.enr.com/articles/55950-us-labor-se...,16-Feb-23,"U.S. Labor Secretary Martin J. Walsh, who is f...",1,0,0,0
2023-02-08,Industry Mostly Hails Biden Speech's Pro-Infra...,https://www.enr.com/articles/55889-industry-mo...,8-Feb-23,Construction and engineering groups praised th...,1,0,0,0
2022-12-21,Pulse: Project News for the Week of December 2...,https://www.enr.com/articles/55646-pulse-proje...,21-Dec-22,Planning\n\nCOLORADO\n\nUbuntu Partners Real E...,1,0,0,0


In [4]:
"""Data cleaning:  Remove any newlines"""
# Convert to list 
data = df.Text.values.tolist()  
# Remove new line characters 
data = [re.sub('\s+', ' ', sent) for sent in data]  
# Remove distracting single quotes 
data = [re.sub("\'", "", sent) for sent in data]
pprint(data[:1])

['Planning ARIZONA Exact Sciences is planning to build a cancer detection '
 'facility in Phoenix. The 250,000-sq-ft building will contain lab, office, '
 'manufacturing and warehouse space on a 17-acre site at Sky Harbor Center. '
 'The project is valued at $50 million or more. Exact Sciences, 441 Charmany '
 'Drive, Madison, Wis., 53719. DR#23-00379030. GEORGIA Kinetic, a fiber '
 'internet provider, plans to carry out a public-private partnership broadband '
 'project in 2024 in Colquitt County. Having already laid 180 miles of fiber '
 'optic cable in the county, this next phase entails laying an additional 440 '
 'miles. Kinetic will use about $21.4 million in state grant money from the '
 'federal government, obtained from the U.S. Coronavirus State and Local '
 'Fiscal Recovery Funds program, part of the federal American Rescue Plan '
 'COVID-19 stimulus package. Kinetic itself will invest $11.1 million and '
 'cover any cost overruns. The communities benefitting from the new '


<>:5: DeprecationWarning: invalid escape sequence '\s'
<>:5: DeprecationWarning: invalid escape sequence '\s'
/tmp/ipykernel_1326671/2842164633.py:5: DeprecationWarning: invalid escape sequence '\s'
  data = [re.sub('\s+', ' ', sent) for sent in data]


In [5]:
"""Tokenization and cleaning up text"""
def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))            #deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['planning', 'arizona', 'exact', 'sciences', 'is', 'planning', 'to', 'build', 'cancer', 'detection', 'facility', 'in', 'phoenix', 'the', 'sq', 'ft', 'building', 'will', 'contain', 'lab', 'office', 'manufacturing', 'and', 'warehouse', 'space', 'on', 'acre', 'site', 'at', 'sky', 'harbor', 'center', 'the', 'project', 'is', 'valued', 'at', 'million', 'or', 'more', 'exact', 'sciences', 'charmany', 'drive', 'madison', 'wis', 'dr', 'georgia', 'kinetic', 'fiber', 'internet', 'provider', 'plans', 'to', 'carry', 'out', 'public', 'private', 'partnership', 'broadband', 'project', 'in', 'in', 'colquitt', 'county', 'having', 'already', 'laid', 'miles', 'of', 'fiber', 'optic', 'cable', 'in', 'the', 'county', 'this', 'next', 'phase', 'entails', 'laying', 'an', 'additional', 'miles', 'kinetic', 'will', 'use', 'about', 'million', 'in', 'state', 'grant', 'money', 'from', 'the', 'federal', 'government', 'obtained', 'from', 'the', 'coronavirus', 'state', 'and', 'local', 'fiscal', 'recovery', 'funds', 'pro

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[:1]]])

In [7]:
"""Remove stopwords, make bigrams, lemmatize"""
# Define function for stopwords, bigrams, trigrams and lemmatization
import math
len_segments = 100

def remove_stopwords(texts):
    return_ls = []
    num_segments = int(math.ceil(len(texts)/len_segments))
    
    for i in range(num_segments):
        start_i = i*len_segments
        end_i = start_i + len_segments
        text_segment = texts[start_i : end_i]
        
        print(f"removing stopwords from {start_i} through {end_i}, segment {i}/{num_segments}")
        return_ls.extend([[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in text_segment])
    return return_ls

def make_bigrams(texts):
    return_ls = []
    num_segments = int(math.ceil(len(texts)/len_segments))
    
    for i in range(num_segments):
        start_i = i*len_segments
        end_i = start_i + len_segments
        text_segment = texts[start_i : end_i]
        
        print(f"making bigrams from {start_i} through {end_i}, segment {i}/{num_segments}")
        return_ls.extend([bigram_mod[doc] for doc in text_segment])
    
    return return_ls

def make_trigrams(texts):
    return_ls = []
    num_segments = int(math.ceil(len(texts)/len_segments))
    
    for i in range(num_segments):
        start_i = i*len_segments
        end_i = start_i + len_segments
        text_segment = texts[start_i : end_i]
        
        print(f"making trigrams from {start_i} through {end_i}, ")
        return_ls.extend([bigram_mod[doc] for doc in text_segment])
    
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    num_segments = int(math.ceil(len(texts)/len_segments))
    for i, sent in enumerate(texts):
        if i % len_segments == 0:
            print(f"lemmztization {i} through {i + len_segments}, segment {i}/{num_segments}")
        
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

removing stopwords from 0 through 100, segment 0/1
making bigrams from 0 through 100, segment 0/1
lemmztization 0 through 100, segment 0/1
[['plan', 'exact', 'science', 'plan', 'build', 'cancer', 'detection', 'facility', 'build', 'contain', 'lab', 'office', 'manufacturing', 'warehouse', 'space', 'sky', 'harbor', 'project', 'value', 'exact', 'drive', 'fiber', 'internet', 'provider', 'plan', 'carry', 'public', 'private', 'partnership', 'broadband', 'already', 'lay', 'mile', 'fiber', 'optic', 'cable', 'county', 'next', 'phase', 'entail', 'lay', 'additional', 'mile', 'kinetic', 'use', 'state', 'grant', 'money', 'federal', 'government', 'obtain', 'coronavirus', 'state', 'local', 'fiscal', 'recovery', 'fund', 'program', 'part', 'federal', 'rescue', 'plan', 'covid', 'stimulus', 'package', 'kinetic', 'invest', 'cover', 'cost', 'overrun', 'community', 'benefit', 'new', 'construction', 'little', 'rock', 'plan', 'begin', 'environmental', 'remediation', 'effort', 'residential', 'property', 'former

In [9]:
Keywords = ["Mental Health"]
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

Keywords_tokenized = list(sent_to_words(Keywords))
Keywords_lemmatized = lemmatization(Keywords_tokenized, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
print(Keywords_lemmatized)

lemmztization 0 through 100, segment 0/1
[['mental', 'health']]


In [10]:
# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 3), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 6), (25, 4), (26, 1), (27, 1), (28, 1), (29, 1), (30, 3), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 3), (39, 1), (40, 3), (41, 4), (42, 2), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 2), (51, 1), (52, 1), (53, 2), (54, 4), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 2), (61, 1), (62, 1), (63, 1), (64, 2), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 3), (80, 1), (81, 2), (82, 3), (83, 1), (84, 3), (85, 2), (86, 3), (87, 1), (88, 1), (89, 7), (90, 3), (91, 1), (92, 1), (93, 1), (94, 2), (95, 1), (96, 1), (97, 1), (98, 2), (99, 1), (100, 1), (101, 2), (102, 1), (103, 1), (104, 1), (105, 2), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1)

In [11]:
"""Readable dictionary"""
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('able', 1),
  ('access', 2),
  ('acre', 1),
  ('acre_site', 1),
  ('action', 1),
  ('additional', 3),
  ('affair', 1),
  ('agreement', 2),
  ('agricultural', 1),
  ('alarm', 1),
  ('already', 1),
  ('also', 1),
  ('animal', 1),
  ('architecture', 1),
  ('array', 1),
  ('assist', 1),
  ('associate', 1),
  ('auditorium', 1),
  ('base', 1),
  ('begin', 2),
  ('benefit', 1),
  ('bhdp', 1),
  ('blvd', 1),
  ('broadband', 1),
  ('build', 6),
  ('building', 4),
  ('cable', 1),
  ('campus', 1),
  ('cancer', 1),
  ('capacity', 1),
  ('carry', 3),
  ('center', 1),
  ('cherrywood', 1),
  ('choose', 1),
  ('closet', 1),
  ('communication', 1),
  ('community', 1),
  ('compact', 1),
  ('complete', 3),
  ('conduct', 1),
  ('consist', 3),
  ('construction', 4),
  ('contain', 2),
  ('contaminate', 1),
  ('convey', 1),
  ('copper', 1),
  ('corner', 1),
  ('coronavirus', 1),
  ('cost', 1),
  ('county', 1),
  ('cover', 2),
  ('covid', 1),
  ('credible', 1),
  ('current', 2),
  ('dam', 4),
  ('date', 1)

In [12]:
"""Build topic model"""
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [13]:
# Print the keyword of topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.036*"say" + 0.018*"project" + 0.013*"also" + 0.012*"infrastructure" + '
  '0.009*"work" + 0.009*"statement" + 0.009*"lumber" + 0.007*"industry" + '
  '0.007*"add" + 0.007*"requirement"'),
 (1,
  '0.001*"say" + 0.000*"work" + 0.000*"construction" + 0.000*"project" + '
  '0.000*"also" + 0.000*"include" + 0.000*"employee" + 0.000*"go" + '
  '0.000*"plan" + 0.000*"year"'),
 (2,
  '0.030*"project" + 0.015*"build" + 0.015*"construction" + 0.014*"value" + '
  '0.012*"building" + 0.011*"state" + 0.011*"plan" + 0.011*"include" + '
  '0.011*"facility" + 0.010*"new"'),
 (3,
  '0.001*"say" + 0.001*"building" + 0.001*"project" + 0.000*"work" + '
  '0.000*"also" + 0.000*"energy" + 0.000*"build" + 0.000*"use" + 0.000*"year" '
  '+ 0.000*"add"'),
 (4,
  '0.035*"say" + 0.014*"state" + 0.014*"team" + 0.012*"project" + 0.011*"byre" '
  '+ 0.011*"industry" + 0.009*"work" + 0.008*"also" + 0.008*"fund" + '
  '0.008*"drone"'),
 (5,
  '0.026*"worker" + 0.021*"say" + 0.018*"recovery" + 0.017*"constru

In [14]:
"""Evaluate topic models"""
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.543061356365854

Coherence Score:  0.37670528998552927


In [15]:
import pandas as pd

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/home/jcm/.local/lib/python3.10/site-packages/pyLDAvis/_prepare.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/jcm/.local/lib/python3.10/site-packages/pyLDAvis/_prepare.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11    -0.133627 -0.024471       1        1  19.867449
15     0.032716 -0.081813       2        1  11.950813
13    -0.107813 -0.069484       3        1   9.206253
12     0.182278 -0.168008       4        1   8.605189
2      0.152216 -0.161531       5        1   8.181006
19    -0.160244 -0.056259       6        1   7.586627
17    -0.128275 -0.053580       7        1   6.115197
18     0.043369  0.031447       8        1   5.102601
4     -0.048266 -0.021380       9        1   4.466101
5     -0.110163 -0.026175      10        1   4.127432
6      0.027020 -0.051380      11        1   3.503957
0     -0.041145 -0.006729      12        1   3.103284
16     0.054464  0.093756      13        1   2.596356
14     0.044766  0.118042      14        1   2.400074
9      0.039484  0.062774      15        1   1.914742
8     -0.020408  0.059636      16        1   1.250365
1      0.043448  0.089100      17        1   0.005727
7      0.043442  0.084994      18        1   0.005707
3      0.042537  0.088500      19        1   0.005687
10     0.044199  0.092562      20        1   0.005432, topic_info=             Term        Freq       Total Category  logprob  loglift
321           say  261.000000  261.000000  Default  30.0000  30.0000
171       project  160.000000  160.000000  Default  29.0000  29.0000
41   construction  126.000000  126.000000  Default  28.0000  28.0000
227         value   57.000000   57.000000  Default  27.0000  27.0000
347        worker   55.000000   55.000000  Default  26.0000  26.0000
..            ...         ...         ...      ...      ...      ...
350          year    0.000296   69.779572  Topic20  -7.9951  -2.5483
226           use    0.000292   49.632008  Topic20  -8.0115  -2.2241
849      employee    0.000291   57.842503  Topic20  -8.0142  -2.3798
160          plan    0.000291   64.069335  Topic20  -8.0152  -2.4831
215         state    0.000290   58.754319  Topic20  -8.0178  -2.3991

[1360 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1667     11  0.717913   absence
682       4  0.157246     abuse
682      10  0.786232     abuse
815       7  0.670397    accept
2043      9  0.686090  accident
...     ...       ...       ...
439      13  0.743100    yorker
930       3  0.100749     young
930       6  0.302247     young
930       7  0.100749     young
930       9  0.503745     young

[2395 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 16, 14, 13, 3, 20, 18, 19, 5, 6, 7, 1, 17, 15, 10, 9, 2, 8, 4, 11])